# COMP1801 Tutorial Week 5 - Classification
*Dr Peter Soar - 2024/25 (based upon tutorials previously made by Dr Jing Wang in 22/23)*

This week we are changing over to the other branch of supervised learning by looking at Classification methods. The general pipeline of finding the best classification model is essentially identical to that of regression. The key differences to be aware of for classification is that there are a different selection of models to test and the evaluation of Classification tasks use different metrics.

For this tutorial, read through and try to understand the text and code examples I have provided (ask your tutor if you have any questions) and there will be a selection of exercises. Attempt these exercises on your own, but do ask your tutor for help if you get stuck.

$\newcommand{\Vec}[1]{\boldsymbol{#1}}$
$\newcommand{\Mat}[1]{\boldsymbol{#1}}$

##0. Do not forget to import all the Python Libraries being used!

In [ ]:
# Import NumPy, which can deal with multi-dimensional arrays such as matrix intuitively.
import numpy as np # A useful package for dealing with mathematical processes, we will be using it this week for vectors and matrices
import pandas as pd # A common package for viewing tabular data
import matplotlib.pyplot as plt # We will be using Matplotlib for our graphs
import seaborn as sns; sns.set()  # for plot styling
import sklearn.datasets # We want to be able to access the sklearn datasets again
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score # required for evaluating classification models
from sklearn.preprocessing import StandardScaler # We will be using the inbuilt preprocessing functions sklearn provides
from sklearn.model_selection import train_test_split # A library that can automatically perform data splitting for us

#1. Classification models
I'm going to start by introducing you to a selection of different classification models that I feel you should be aware of. This is far from an exhaustive list of all classifiers out there, and do feel free to look through the [sklearn documentation](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) and explore the other models available.

Which of these models you try to implement with the coursework is up to you - there may be some you feel are more appropriate to the problem so you focus on them, or you might decide to experiment with all of them. There is no definitive right or wrong model to use, but I do expect justifications to support whichever models you decide to experiment with (including any models not on this list).

I'm not going to be providing a full pipeline example for this section - just a brief example demonstrating the behaviour on the entire breast cancer dataset from sklearn and evaluated with naive accuracy.



##1.1 Logistic Regression

Logistic regression is in many ways the form of classification that follows on the most intuitively from the linear regression models we have been exploring so far. Despite the name, Logistic Regression is a classification model, that works by regressing on the *probability* of a data point being in a given class, which we can then use to decide what class the point belongs in.

The hypothesis function of the logistic regression model is given as follows:
$$
h_{\Vec{\theta}} (\Vec{x}^{(i)}) =
\begin{cases}
0 & \textrm{if ${\Vec{x}^{(i)}} \Vec{\theta} < 0$,} \\
1 & \textrm{if ${\Vec{x}^{(i)}} \Vec{\theta} > 0$.}
\end{cases}
$$

The cost function of the logistic regression is given as follows:
$$
J_{C} (\Vec{\theta}) = L (\Vec{\theta}) + R_{C} (\Vec{\theta}),
$$
where the loss function $L$ is given by
$$
L (\Vec{\theta})
=
- \frac{1}{m} \sum_{i=0}^{m-1} \ln \varsigma (y^{(i)} \cdot {\Vec{x}^{(i)}}\Vec{\theta})
$$
<!-- if the class weight is not considered,
$$
L (\Vec{\theta})
:=
- \sum_{i=0}^{m-1} \frac{1}{2 m_{y^{(i)}}} \ln \varsigma (y^{(i)} \cdot {\Vec{x}^{(i)}}^{\top} \Vec{\theta})
$$
and
$$
R_{C} (\Vec{\theta})
:=
\frac{1}{2 m C} \sum_{j=1}^{n-1} (\theta_{j})^2.
$$ -->

Here,
- $m$ is the number of data points.
<!-- - $m_{y}$ is the number of data points whose target value is $y$. -->
- $L$ is the loss function that gives the cross entropy,
- $R_{C}$ is the regularization function that gives a $\ell^2$ regularization term,
- $C > 0$ is the **inverse** regularization weight. We use the inverse one just for historical reasons.

  - If $C$ is **small**, the regularization is strong (corresponding to a large regularization weight), and overfitting is strongly avoided, but may cause underfitting.
  - If $C$ is **large**, the regularization is weak (corresponding to a small regularization weight). In this case the model is close to the original model, and it may cause overfitting.

The key settings/hyperparameters to consider for Logistic Regression are:
- `C` - The inverse Regularisation Weight
- `penalty` - the regulation type used (note: all do not work with all solvers, see documentation)
  - `l1` - applies a L1 penalty (as in our Lasso Regression)
  - `l2` - applies a L2 penalty (as in our Ridge Regression)
  - `elasticnet` - a method that applies both an L1 and L2 penalty to the optimisation.
- `solver` - the solver used to obtain the solution (Options: `lbfgs`, `liblinear`, `newton-cg`, `newton-cholesky`, `sag`, `saga`)
- `random_state` - Allows you to recreate the same solution by generating the same quasi-random case (as the solution is found by optimisation, not direct solution).

While these are the main ones I would expect you to explore at this stage, there are some more you can look into by reading the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression) on sklearn.

**Task**: try changing some of these settings in the case below to see if you can understand the impact they are having on the results.



In [ ]:
# Import the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
################
# Load Data
################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)
# Convert the data to `np.array`
# Note we are using teh whole dataset - not a robust pipeline!
X = np.array(X_pd[['mean radius', 'mean texture']]) # just select two features
y = np.array(y_pd)

################
# Initialisation
################
model = LogisticRegression() # Create our prediction model object
# See below an example of some of the key settings
#model = LogisticRegression(penalty='l1', solver = 'liblinear', C=0.01, random_state=40)
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########

# It is almost always expected that you scale your data as Logistic regression is regularised
scaler.fit(X) # Fit the standardisation object on the polynomial feature matrix
X = scaler.transform(X) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X, y) # fit our logistic regression model - 'train' the model

############
# Evaluation
############
# Evaluate performance of the training data
y_pred = model.predict(X) # Use our fitted model to make a prediction

# Calculate accuracy - note this can often be a naive classification accuracy criterion!
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Plot the decision boundary
h = .02  # step size in the mesh
# boundaries for the grid
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
# Create mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
# Use model to plot the decision boundary for all points
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
# Plot observed points and add text
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title(f'Logistic Regression Classifier on Breast Cancer Dataset')
plt.xlabel('mean radius')
plt.ylabel('mean texture')
plt.show()

##1.2 Support Vector Machines

Support Vector Machines are another very popular machine learning method (which can be used for regression and classification). The mathematics behind them are quite complicated and different to what we have been looking at in our linear models - using objects called 'kernels' to draw the line that best separates the classes, rather than regressing on the probability like our Logistic Regression (as such, we don't generally get a probability estimate from the SVM).

There are a few key settings you should consider for SVMs:
- `C` - The inverse Regularisation Weight (there are no options of the type of regularisation for SVM).
- `kernel` - the type of kernel the SVM model uses to fit the lines
  - `linear` - Draws a straight line
  - `poly` - Same as manually fitting polynomial features and using the linear kernel (though doing it this manual way would be computationally slower).
    - `degree` - Choose maximum polynomial degree
  - `rbf` - Uses Kernel trick to measure similarity between data points in infinite dimensions and classify based on this.
  - `sigmoid` - Creates a complex sigmoid shaped irregular boundary. Only generally useful for very specific datasets as is often does not generalise well.
- `random_state` - Allows you to recreate the same solution by generating the same quasi-random case.

**Task**: try changing some of these settings/hyperparameters in the case below to see if you can understand the impact they are having on the results.

There are some other settings/hyperparameters you could experiment with that you can find in the sklearn [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC).



In [ ]:
# Import the Support Vector Machine Classification Model
from sklearn.svm import SVC
################
# Load Data
################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)
# Convert the data to `np.array`
# Note we are using teh whole dataset - not a robust pipeline!
X = np.array(X_pd[['mean radius', 'mean texture']]) # just select two features
y = np.array(y_pd)

################
# Initialisation
################
model = SVC() # Create our prediction model object
# See below an example of some of the key settings
#model = SVC(C=1,kernel='rbf',random_state=40)
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########

# It is almost always expected that you scale your data as SVM is regularised
scaler.fit(X) # Fit the standardisation object on the polynomial feature matrix
X = scaler.transform(X) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X, y) # fit our SVM - 'train' the model

############
# Evaluation
############
# Evaluate performance of the training data
y_pred = model.predict(X) # Use our fitted model to make a prediction

# Calculate accuracy - note this can often be a naive classification accuracy criterion!
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Plot the decision boundary
h = .02  # step size in the mesh
# boundaries for the grid
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
# Create mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
# Use model to plot the decision boundary for all points
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
# Plot observed points and add text
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title(f'SVM Classifier on Breast Cancer Dataset')
plt.xlabel('mean radius')
plt.ylabel('mean texture')
plt.show()

##1.3 Naive Bayes

Naive Bayes is an interesting case - it is purely probabilistic and doesn't really have any hyperparameters to tune. It's non-parametric so it doesn't case about feature scaling. One of it's key assumptions is that all features are independent (statistically), which is often not the case. Regardless, Naive Bayes is often surprisingly good at getting classification right (though its prediction of the probability should not be taken too seriously).

It is quite unlikely that naive Bayes will be your 'best' model in almost any case, but it is very quick and easy to fit and can act as a good baseline model to compare against. And sometimes it may be good enough for what you are trying to do!

You can read the sklearn [documentation](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) if you are interested in learning more about how Naive Bayes work (and there are other types you can experiment with).



In [ ]:
# Import the Naive Bayes Classification Model
from sklearn.naive_bayes import GaussianNB
################
# Load Data
################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)
# Convert the data to `np.array`
# Note we are using the whole dataset - not a robust pipeline!
X = np.array(X_pd[['mean radius', 'mean texture']]) # just select two features
y = np.array(y_pd)

################
# Initialisation
################
model = GaussianNB() # Create our prediction model object
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########

# For Naive Bayes, this is a case where feature scaling doesn't matter!
#scaler.fit(X) # Fit the standardisation object on the polynomial feature matrix
#X = scaler.transform(X) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X, y) # fit our Naive Bayes - 'train' the model

############
# Evaluation
############
# Evaluate performance of the training data
y_pred = model.predict(X) # Use our fitted model to make a prediction

# Calculate accuracy - note this can often be a naive classification accuracy criterion!
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Plot the decision boundary
h = .02  # step size in the mesh
# boundaries for the grid
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
# Create mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
# Use model to plot the decision boundary for all points
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
# Plot observed points and add text
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title(f'Naive Bayes Classifier on Breast Cancer Dataset')
plt.xlabel('mean radius')
plt.ylabel('mean texture')
plt.show()

##1.4 k-Nearest Neighbours

You should already be somewhat familiar with the concept of Knn for regression - and the principles are exactly the same for classification (in fact I would say they are even more intuitive). Using the training date, the Knn classifier can check what the nearest neighbours are for each point needing to be predicted, whichever class contains the most neighbours will be the predicted class.

While there are some other options you can read about in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier), the key hyperparameter of interest is the `n_neighbors`.

**Task**: try experimenting with the `n_neighbors` to see how it impacts the prediction (but remember that we are not evaluating this data properly using a test dataset!)

In [ ]:
# Import the Knn Classification Model
from sklearn.neighbors import KNeighborsClassifier
################
# Load Data
################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)
# Convert the data to `np.array`
# Note we are using the whole dataset - not a robust pipeline!
X = np.array(X_pd[['mean radius', 'mean texture']]) # just select two features
y = np.array(y_pd)

################
# Initialisation
################
model = KNeighborsClassifier(n_neighbors=3) # Create our prediction model object
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########

# You probably still want to scale your features for best behaviour.
scaler.fit(X) # Fit the standardisation object on the polynomial feature matrix
X = scaler.transform(X) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X, y) # fit our Knn - 'train' the model

############
# Evaluation
############
# Evaluate performance of the training data
y_pred = model.predict(X) # Use our fitted model to make a prediction

# Calculate accuracy - note this can often be a naive classification accuracy criterion!
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Plot the decision boundary
h = .2  # step size in the mesh
# boundaries for the grid
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
# Create mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
# Use model to plot the decision boundary for all points
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
# Plot observed points and add text
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title(f'Knn Classifier on Breast Cancer Dataset')
plt.xlabel('mean radius')
plt.ylabel('mean texture')
plt.show()

##1.5 Decision Tree

Decision Trees are a powerful non-parametric method that you should have encountered already for regression problems. This looks at your variables and asks a series of yes/no questions which it uses to split into a 'tree' of conditions and target predictions. Being non-parametric, the model does not care about feature scaling.

Decision trees do have a lot of variables that can (and should) be experimented with to get a model that is picking up realistic behaviour in the data:

*   `max_depth` which dictates how many levels the tree can have (where a 'deeper' tree can include more complexity as more yes/no questions are asked at each level).
*   `min_samples_split` How many samples need to exist at a node to split the data with a further yes/no question
*   `min_samples_leaf` How many samples need to exist on each branch after a split for a split to be considered.

You can find more information about these variables (and more) in the sklearn [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier).

**Task**: Experiment with these parameters and see what it does to your prediction. Again, remember that our pipeline here is not performing a robust evaluation and that decision trees can be quite prone to overfitting!


In [ ]:
# Import the Decision Tree Classification Model
from sklearn.tree import DecisionTreeClassifier
################
# Load Data
################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)
# Convert the data to `np.array`
# Note we are using the whole dataset - not a robust pipeline!
X = np.array(X_pd[['mean radius', 'mean texture']]) # just select two features
y = np.array(y_pd)

################
# Initialisation
################
model = DecisionTreeClassifier() # Create our prediction model object
#model = DecisionTreeClassifier(max_depth = 3, min_samples_split = 5, min_samples_leaf = 5)
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########

# For Decision Trees, this is a case where feature scaling doesn't matter!
#scaler.fit(X) # Fit the standardisation object on the polynomial feature matrix
#X = scaler.transform(X) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X, y) # fit our Decision Tree - 'train' the model

############
# Evaluation
############
# Evaluate performance of the training data
y_pred = model.predict(X) # Use our fitted model to make a prediction

# Calculate accuracy - note this can often be a naive classification accuracy criterion!
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Plot the decision boundary
h = .2  # step size in the mesh
# boundaries for the grid
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
# Create mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
# Use model to plot the decision boundary for all points
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
# Plot observed points and add text
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title(f'Decision Tree Classifier on Breast Cancer Dataset')
plt.xlabel('mean radius')
plt.ylabel('mean texture')
plt.show()

##1.6 Random Forests

Random forests are what are known as an 'ensemble model', meaning that they are the result of fitting multiple models (in this case Decision Tree models) on subsamples of the data, with the results from all of these runs being averaged to get the best model overall and try to tackle overfitting (though random forests are still very prone to overfitting). Being non-parametric, the model does not care about feature scaling.

You can essentially think of this as an advanced form of decision tree - they use all of the same parameters outlined above, with the addition of `n_estimators`, which controls how many Decision Tree Models are created and combined for the final random forest model. As the name implies, these models are made using random samples, so for replicating your tests you should also generally set a `random_state`.

One downside of random forests are that they are 'black box' models that are not really interpretable, whereas arguably the biggest benefit of decision trees are that they are very easy to interpret and understand why they have made a choice to classify one way or the other.

Read more about Random forests in the [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) documentation.

**Task**: Briefly experiment with the hyperparameters to see if you can understand how this is impacting the results.



In [ ]:
# Import the Random Forest Classification Model
from sklearn.ensemble import RandomForestClassifier
################
# Load Data
################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)
# Convert the data to `np.array`
# Note we are using the whole dataset - not a robust pipeline!
X = np.array(X_pd[['mean radius', 'mean texture']]) # just select two features
y = np.array(y_pd)

################
# Initialisation
################
model = RandomForestClassifier() # Create our prediction model object
#model = RandomForestClassifier(n_estimators=50, max_depth = 3, min_samples_split = 5, min_samples_leaf = 5, random_state = 0)
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########

# For Random Forest, this is a case where feature scaling doesn't matter!
#scaler.fit(X) # Fit the standardisation object on the polynomial feature matrix
#X = scaler.transform(X) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X, y) # fit our Random Forest - 'train' the model

############
# Evaluation
############
# Evaluate performance of the training data
y_pred = model.predict(X) # Use our fitted model to make a prediction

# Calculate accuracy - note this can often be a naive classification accuracy criterion!
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Plot the decision boundary
h = .2  # step size in the mesh
# boundaries for the grid
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
# Create mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
# Use model to plot the decision boundary for all points
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
# Plot observed points and add text
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title(f'Random Forest Classifier on Breast Cancer Dataset')
plt.xlabel('mean radius')
plt.ylabel('mean texture')
plt.show()

# 2. Classification implementation

In this section, I want to provide you with some more realistic examples of how we would aim to fit a classification model in practice, with a robust evaluation and model selection pipeline.

## 2.1 Full ML pipeline (Classification)

Here is a repeat of the ML pipeline that was presented in the previous weeks lecture - as said, the general pipeline for fitting a supervised learning ML model is essentially the same whether you are using Regression or classification, so in the pipeline I am just highlighting in red the few differences you have to consider for classification models - as you can see these are quite minimal!

General pipeline:
- Data loading
  - Ensure you perform and Exploratory Data Analysis (EDA) and clean your data before going any further when dealing with real data.
  - Convert your data into arrays that your ML model can interface with.
  - Split your data into the Train:Validation:Test sets.
- Initialization
  - Initialize the preprocessor(s) and the <font color='red'>**machine learning model objects** (You will have a different selection of models to consider during classification - see above)</font>.
- Model Selection
  - Identify the different solutions you want to try: this could involve different model types (e.g. Linear Regression, Lasso, Ridge, Decision tree, Neural Network, etc), preprocessing methods, feature selection/engineering and hyperparameter selection.
  - Training (do this for **all** models and hyperparameters)
    - **Fit the preprocessor(s)** to the raw training feature matrix.
    - **Transform (Preprocess)** the raw training feature matrix into the pre-processed training feature matrix.
    - **Fit (Train) the prediction model** using the data pair of the pre-processed training feature matrix and target column vector.
    - Recording the performance of the prediction on training data is often also helpful for identifying overfitting.

  - Validation (for **all** models and hyperparameters)
    - **Transform (Preprocess)** the raw validation feature matrix into the pre-processed validation feature matrix using **the preprocessor(s)** (**Do NOT fit any preprocessor to the validation feature matrix**).
    - **Predict** using the pre-processed validation feature matrix to get a predicted target column.
    - <font color='red'>**Evaluate** the prediction performance on the validation data by Accuracy Metrics / Confusion Matrix</font>

  - Continue training and validation until you have a model (or selection of models) you feel are potentially good enough to offer a robust solution to the provided prediction problem.

- Test (for the model and hyperparameters selected by the validation process)
  - **Transform (Preprocess)** the raw test feature matrix into the pre-processed test feature matrix by **the preprocessor(s)** (**Do NOT fit any preprocessor to the test feature matrix**).
  - **Predict** on the pre-processed test feature matrix to get a predicted target column.
  - <font color='red'>**Evaluate** the prediction performance on the test data by Accuracy Metrics / Confusion Matrix</font>

If after the test step your model is not providing good enough solutions on the test data, then you will have to go back to rethink how you can create a more robust solution less prone to overfitting.

##2.2 Binary Classification Example


Classification can be thought of as coming in two types - binary and multi-class. While very similar in how we approach them (and most models can happily deal with both without changing and variables). However, there are still enough different considerations that it is worth considering them separately.

I'm going to start with the slightly simpler case of binary classification, for which we'll be using another `sklearn` dataset about breast cancer.

In [ ]:
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)
Xy_df = pd.concat([X_pd, y_pd], axis=1)

display(Xy_df)

# Convert the data to `np.array`
# We're just going to use two features for now.
X_raw = np.array(X_pd[['mean radius', 'mean texture']])
# Note that in the dataset 0 = Malignant and 1 = Benign, but we're going to switch that around going forward
# so that the 'positive' class Malignant = 1
y = np.array(1.0-y_pd)

We want to robustly split our data into three sets for classification as well. Note that really we always want to shuffle our classification data for tabular problems like this.

In [ ]:
# Split the data into non-test/test data
# While we have 20640 pairs of a feature and target, we use 20% only for the testing, not so we hold back the other 80% for training and validation
# Generally we always want to shuffle data for classification.
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=0.20, shuffle=True, random_state=0)

# Split the non-test data into non-test/test data
# We use 25% of the remaining non-test data only for the validation set, leaving the rest for training
# In the end, the splitting ratio will be 6:2:2 for the training, validation, and test data.
# Generally we always want to shuffle data for classification.
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=0.25, shuffle=True, random_state=0)

We can see that initialisation, training and validation follow the same steps. Note that the model we are initialising here is a `LogisticRegression` model, though for now we are just using the default values.

In [ ]:
################
# Initialisation
################
model = LogisticRegression() # Create our prediction model object
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########

scaler.fit(X_train_raw) # Fit the standardisation object on the polynomial feature matrix
X_train = scaler.transform(X_train_raw) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X_train, y_train) # fit our logistic regression model - 'train' the model

############
# Validation
############

# Preprocess validation dataset using preprocessor objects for on the training data
# Do NOT refit the preprocessors
X_valid = scaler.transform(X_valid_raw) # Standardise the validation data

# We select the best hyperparameters based on the performance of our model on the validation dataset
y_pred_valid = model.predict(X_valid) # Use our fitted logistic regression model to make a prediction based on teh validation dataset

When we only have two features, we can visualise the prediction by plotting the decision boundary (which says what category the model predicts fo every possible point in an area), and by plotting the points from each class over this we can see where the model is failing.

In [ ]:
# Plot outputs
# Create a meshgrid to plot the decision boundary
x_min, x_max = X_valid[:, 0].min() - 1, X_valid[:, 0].max() + 1
y_min, y_max = X_valid[:, 1].min() - 1, X_valid[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))

# Predict the class labels for the points in the meshgrid
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Create a scatter plot of the data points

# Plot the decision boundary
plt.contourf(xx, yy, Z, alpha=0.35, cmap = 'bwr')
X_neg = X_valid[y_valid==0, :]
X_pos = X_valid[y_valid==1, :]
plt.scatter(X_neg[:, 0], X_neg[:, 1], color='blue', edgecolors='black', label='Benign')
plt.scatter(X_pos[:, 0], X_pos[:, 1], color='red', edgecolors='black', label='Malignant')


plt.xlabel('mean radius')
plt.ylabel('mean texture')

# Show the plot
plt.title(f'Logistic Regression Classifier on Breast Cancer Dataset (Validation)')
plt.legend()
plt.show()

### Classification accuracy metrics
One we have our the model fit on the validation data, we can explore various classification accuracy metrics to see how good our model is. While there is no harm in checking all of these, there is generally one metric in particular you care the most about maximising, which is generally based both on your problem (for example we generally care more about recall if a false negative is really bad - e.g. missing if someone has cancer) or the distribution of your data (if your data is well balanced then the accuracy may be a good metric).

Here are some of the common classification metrics:
- **Accuracy** defined by
$$
\mathrm{Acc} := \frac{\#\{i|\hat{y}^{(i)} \ne y^{(i)}\}}{m},
$$
calculated by `sklearn.metrics.accuracy_score`.

- **Precision** for category $y$ defined by
$$
\mathrm{Pre}_{y} := \frac{\mathrm{TP}_{y}}{\mathrm{TP}_{y} + \mathrm{FP}_{y}},
$$
calculated by `sklearn.metrics.precision_score`.

- **Recall** for category $y$ defined by
$$
\mathrm{Rec}_{y} := \frac{\mathrm{TP}_{y}}{\mathrm{TP}_{y} + \mathrm{FN}_{y}},
$$
calculated by `sklearn.metrics.recall_score`.

- **F1 score** for category $y$ defined by
$$
\mathrm{F1}_{y} := \frac{1}{\frac{1}{\mathrm{Pre}_{y}} + \frac{1}{\mathrm{Rec}_{y}}},
$$
calculated by `sklearn.metrics.f1_score`  with `average='binary'` (default).

Advanced:

- **Balanced accuracy** defined by
$$
\mathrm{BAcc} := \frac{1}{\#\mathcal{Y}} \sum_{y \in \mathcal{Y}} \mathrm{Rec}_{y},
$$
calculated by `sklearn.metrics.balanced_accuracy_score`.

- **Macro-averaged F1 score** defined by
$$
\mathrm{MacroF1} := \frac{1}{\#\mathcal{Y}} \sum_{y \in \mathcal{Y}} \mathrm{F1}_{y},
$$
calculated by `sklearn.metrics.f1_score` with `average='macro'`.

- **Weighted-averaged F1 score** defined by
$$
\mathrm{WeightF1} := \sum_{y \in \mathcal{Y}} \frac{m_y}{m} \mathrm{F1}_{y},
$$
calculated by `sklearn.metrics.f1_score` with `average='macro'`.

Here, $\mathcal{Y}$ is the set of possible target values, $\#$ indicates the number of elements in a set, $m$ is the number of data points, $m_{y} := \#\{i|y^{(i)} = y.\}$ is the number of data points whose actual target is $y$. The true positives $\mathrm{TP}_{y}$, false positives $\mathrm{FP}_{y}$, false negatives $\mathrm{FN}_{y}$, and "true" negatives $\mathrm{TN}_{y}$ are defined by
$$
\mathrm{TP}_{y} := \#\{i|y^{(i)} = y \textrm{, and } \hat{y}^{(i)} = y.\},
$$
$$
\mathrm{FP}_{y} := \#\{i|y^{(i)} \ne y \textrm{, and } \hat{y}^{(i)} = y.\},
$$
$$
\mathrm{FN}_{y} := \#\{i|y^{(i)} = y \textrm{, and } \hat{y}^{(i)} \ne y.\},
$$
$$
\mathrm{TN}_{y} := \#\{i|y^{(i)} \ne y \textrm{, and } \hat{y}^{(i)} \ne y.\}.
$$

Also, sklearn has various functions for plotting the confusion matrix and classification reports, which are good for allowing you to evaluate your model at a glance.

In [ ]:
# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid)))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid)))
print('F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))


Finally, once we've happy that we've got a good model according to the validation data, we can go ahead with the final evaluation on the test data.

In [ ]:
############
# Test
############

# Preprocess Test dataset using preprocessor objects for on the training data
# Do NOT refit the preprocessors
X_test = scaler.transform(X_test_raw) # Standardise the validation data

# We select the best hyperparameters based on the performance of our model on the validation dataset
y_pred_test = model.predict(X_test) # Use our fitted linear regression model to make a prediction based on teh validation dataset

# Plot outputs
# Create a meshgrid to plot the decision boundary
x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))

# Predict the class labels for the points in the meshgrid
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.35, cmap = 'bwr')
X_neg = X_test[y_test==0, :]
X_pos = X_test[y_test==1, :]
plt.scatter(X_neg[:, 0], X_neg[:, 1], color='blue', edgecolors='black', label='Benign')
plt.scatter(X_pos[:, 0], X_pos[:, 1], color='red', edgecolors='black', label='Malignant')
plt.xlabel('mean radius')
plt.ylabel('mean texture')
plt.title(f'Logistic Regression Classifier on Breast Cancer Dataset (Test)')
plt.legend()
plt.show()

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(sklearn.metrics.classification_report(y_test, y_pred_test))


### Note on visualising higher order multivariable classification models

In the solutions so far I've been including plots of the class points over the decision boundary, which I find to be a nice way of visualising how the classification model is drawing the lines to make its decisions. See an example of this in the code cell below - the background colour represents the decision boundaries drawn by the classifier, while the labelled points show what class a data point belongs to. We can visually see that our model does a pretty good job, but we have some crossover of points near the boundary.

In [ ]:
# Create a meshgrid to plot the decision boundary
x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))
# Predict the class labels for the points in the meshgrid
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.35, cmap = 'bwr')
X_neg = X_test[y_test==0, :]
X_pos = X_test[y_test==1, :]
plt.scatter(X_neg[:, 0], X_neg[:, 1], color='blue', edgecolors='black', label='Benign')
plt.scatter(X_pos[:, 0], X_pos[:, 1], color='red', edgecolors='black', label='Malignant')
plt.xlabel('mean radius')
plt.ylabel('mean texture')
plt.title(f'Logistic Regression Classifier on Breast Cancer Dataset (Test)')
plt.legend()
plt.show()

But this only works because we are only using two features, we would be unable to draw this decision boundary in more than two dimensions (well, this isn't strictly true, with a 3D plot we could draw the decision surface of 3 features, but we still would then be unable to visualise the decision shape if we had any more features). While there is a similar problem in regression, at least we can get some information from plotting the predicted vs the observed values in a 2D plot (even if we are missing some information). For classification though, without the decision boundary it is very hard to intuit how well your model is doing visually.

As such, for most classification models with more features I would just be expecting to see the accuracy metrics/confusion matrix.

##2.3 Multi-class classification Example

In this section I am going to give you an example of how to fit a multi-class classification model, using the 'Iris' dataset on `sklearn`.

As most things are the same I'm just going to be highlighting how a multi-class problem requires some different considerations.

In [ ]:
# Load the house price dataset
X_pd, y_pd = sklearn.datasets.load_iris(return_X_y=True, as_frame=True)
Xy_df = pd.concat([X_pd, y_pd], axis=1)

display(Xy_df)

# Convert the data to `np.array`
X_raw = np.array(X_pd[['sepal length (cm)', 'sepal width (cm)']])
y = np.array(y_pd)


Likewise, splitting, preprocessing and fitting the model is the same.

In [ ]:
# Split the data into non-test/test data
# While we have 20640 pairs of a feature and target, we use 20% only for the testing, not so we hold back the other 80% for training and validation
# Generally we always want to shuffle data for classification.
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=0.20, shuffle=True, random_state=0)

# Split the non-test data into non-test/test data
# We use 25% of the remaining non-test data only for the validation set, leaving the rest for training
# In the end, the splitting ratio will be 6:2:2 for the training, validation, and test data.
# Generally we always want to shuffle data for classification.
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=0.25, shuffle=True, random_state=0)

################
# Initialisation
################
model = LogisticRegression() # Create our prediction model object
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########

scaler.fit(X_train_raw) # Fit the standardisation object on the polynomial feature matrix
X_train = scaler.transform(X_train_raw) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X_train, y_train) # fit our linear regression model - 'train' the model

############
# Validation
############

# Preprocess validation dataset using preprocessor objects for on the training data
# Do NOT refit the preprocessors
X_valid = scaler.transform(X_valid_raw) # Standardise the validation data

# We select the best hyperparameters based on the performance of our model on the validation dataset
y_pred_valid = model.predict(X_valid) # Use our fitted linear regression model to make a prediction based on teh validation dataset

We can see below how our confusion matrix has been changed - as you would expect, there are now three groups to correspond to the three classes. This doesn't really change the interpretation, as we still have correct and incorrect predictions.

The only thing to get your head around is that we don't really have a 'positive' class any more, so any notion of false positives/negatives applies only *on a the level of the class we are currently looking at* rather than a statement for the entire model prediction.

Likewise, our classification report has a row for each class now, so again we have to consider the classes individually from these metrics.

In [ ]:
# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(sklearn.metrics.accuracy_score(y_valid, y_pred_valid)))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['Setosa', 'Versicolour', 'Virginica']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))


Finally we come to testing, and again things remain the same - but now we can see our decision boundary has three sections.

The main thing I want you to note is that we now HAVE to average some of our classification metrics (types of `accuracy` are unaffected). In binary classification we can have a 'positive' class that the model considers for the precision/recall/f1, but as mentioned above we don't have this option for multi class problems.

In [ ]:
############
# Test
############

# Preprocess Test dataset using preprocessor objects for on the training data
# Do NOT refit the preprocessors
X_test = scaler.transform(X_test_raw) # Standardise the validation data

# We select the best hyperparameters based on the performance of our model on the validation dataset
y_pred_test = model.predict(X_test) # Use our fitted linear regression model to make a prediction based on teh validation dataset

# Create a meshgrid to plot the decision boundary
x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))

# Predict the class labels for the points in the meshgrid
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the decision boundary
plt.contourf(xx, yy, Z, alpha=0.35, cmap = 'jet')
# Create a scatter plot of the data points
plt.xlabel('Sepal Length')
plt.ylabel('Sepal Width')
X_0 = X_valid[y_valid==0, :]
X_1 = X_valid[y_valid==1, :]
X_2 = X_valid[y_valid==2, :]
plt.scatter(X_0[:, 0], X_0[:, 1], color='blue', edgecolors='black', label='Setosa')
plt.scatter(X_1[:, 0], X_1[:, 1], color='green', edgecolors='black', label='Versicolour')
plt.scatter(X_2[:, 0], X_2[:, 1], color='red', edgecolors='black', label='Virginica')

# Show the plot
plt.title(f'Logistic Regression Classifier on Iris Dataset')
plt.legend()
plt.show()

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(sklearn.metrics.accuracy_score(y_test, y_pred_test)))
######## Note we have to average some of our metrics now!
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test, average='macro')))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test, average='macro')))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='weighted')))
#######################################################
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(sklearn.metrics.classification_report(y_test, y_pred_test))


## 2.4 Hyperparameter selection

As I keep stressing, there really isn't a difference in the model selection process between classification and regression - other than the models you can use and the evaluation metrics.

As such, all the techniques and examples I have already shown you relating to hyperparameter selection and model selection still hold true for these questions. As such, you should still split your data into thtee sets (or use cross validation) and you can use any combination of manual experimentation, hand coded loops and grid searches to help you optimise your hyperparameters and choose the best model.

But just to give you a directly comparable example, below I am providing an example of how we could optimise our inverse regression weight for a logistic regression model. If you still don't feel confident with the principles of model selection, do make sure you read through this code (and comments) carefully to make sure you understand what is being done, do ask your tutor if you have any doubts or questions.

In [ ]:
#####################
# Load and split data
#####################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
# note I'm selecting 4 features here - which is more than in our previous examples.
X_raw = np.array(X_pd[['mean radius', 'mean texture', 'mean perimeter', 'mean area']])
# Note that in the dataset 0 = Malignant and 1 = Benign, but we're going to switch that around going forward
# so that the 'positive' class Malignant = 1
y = np.array(1.0-y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=1/10, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/9, shuffle=True, random_state=0)


###################################
# Initialise Model and preprocessors
###################################

C_indices = np.arange(20) # Get a list of 1-20
Cs = 10.0 ** ( C_indices-10) # Define a list of Cs by taking indices from 10 to -10
# Get the F1 score arrays
# Note that we could be monitoring any of the other metrics to choose our model.
#I have chosen f1 here as it give a balanced penalisation of accuracy and recall, but for your case you may care about maximising another metric
f1_train_array = np.full([len(Cs)], np.nan)
f1_valid_array = np.full([len(Cs)], np.nan)

#Initialise our scaler object - remember that for Logistic regression we should always scale features
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_valid = scaler.transform(X_valid_raw)

# Loop to test C values
for C_index, C in zip(C_indices, Cs):
  # train
  model = LogisticRegression(C=C) # fits our Logistic Regression model with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  f1_train = f1_score(y_train, y_pred_train)

  # Validation
  y_pred_valid = model.predict(X_valid)
  f1_valid = f1_score(y_valid, y_pred_valid)

  # Store f1s for this iteration
  f1_train_array[C_index] = f1_train
  f1_valid_array[C_index] = f1_valid
  print(f'C: {C}, f1 score: {f1_valid}.')

# Plot our graph of F1 for training and validation
plt.plot(Cs, f1_train_array, label='F1 score on training dataset')
plt.plot(Cs, f1_valid_array, label='F1 Score on validation dataset')
plt.xlabel(r'Regularization weights $C$')
plt.ylabel('F1 Score')
plt.title(r'Regularization weights $C$ and F1 Score')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing C (highest f1)
best_c_index = np.nanargmax(f1_valid_array)
best_C = Cs[best_c_index]
print(f'\nThe best C:', best_C)

######
# Test
######
model = LogisticRegression(C=best_C) # fits our Logistic Reg model with the best weight
model.fit(X_train, y_train)
# Preprocess test data
X_test = scaler.transform(X_test_raw)
# Predict test data
y_pred_test = model.predict(X_test)

##########
# Evaluate
##########

# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# Accuracy metrics - 1 is an perfect prediction
print('\nAccuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test)))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test)))
print('F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test)))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test)))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'\nConfusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(f'\nClassification Report:')
print(sklearn.metrics.classification_report(y_test, y_pred_test))


##2.5 Comparison with a naive baseline

One final thing I would say for evaluating your classification model, is that a good final check that your model is behaving well is to see how it compares to what we would call a 'naive baseline'. Generally I would recommend doing this as a final check after you have evaluated the test data as a bit of a 'sanity check'.

This can be done using a simpler model (such as Naive Bayes or K-nearest neighbours), but if often done by just classifying by a very basic rule without even looking at the data. If your model does not outperform this (or is very similar) you have to consider if your model is finding any real patterns in the data.

The two most common ways I see this being done is just classifying all things as the same class (generally the majority class if there is an unbalance or the positive class), or by randomly selecting a class.

Lets try some examples of this on our breast cancer model fitted above.

In [ ]:
# Here we are just going to assume that every point is positive (malignant)
# While this does give us perfect recall, I think we agree that telling 60% of people
# incorrectly that they have cancer is not ideal!

y_pred_test = np.ones_like(y_test)

##########
# Evaluate
##########

# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# Accuracy metrics - 1 is an perfect prediction
print('\nAccuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test)))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test)))
print('F1 Score: {:.4f}'.format(f1_score(y_test, y_test)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test)))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test)))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'\nConfusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(f'\nClassification Report:')
print(sklearn.metrics.classification_report(y_test, y_pred_test))

In [ ]:
# Here I'm generating 0 or 1 randomly for each observation
# Note I've set the numpy random seed so this is replicable
# The performance of this is pretty rubbish - which is good!
np.random.seed(1234)
y_pred_test = np.random.randint(2, size=len(y_test))

##########
# Evaluate
##########

# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# Accuracy metrics - 1 is an perfect prediction
print('\nAccuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test)))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test)))
print('F1 Score: {:.4f}'.format(f1_score(y_test, y_test)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test)))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test)))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'\nConfusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(f'\nClassification Report:')
print(sklearn.metrics.classification_report(y_test, y_pred_test))

Based on both of these, I thing that we can be confident that our model is performing significantly better than a naive baseline - which is what we want to see!

# 3. Exercises

We've reached the stage now where for the most part in the exercises, rather than having a single task I want you to experiment with your models to see how good an accuracy you can get for the problem, using all of the tools I have provided you with so far. So when attempting these exercises to fit the models (and when doing your coursework!) make sure you consider the following (non exhaustive) list of aspects to consider:
- Data splitting (consider cross validation)
- Preprocessing steps
- Features used (will you remove any manually, or leave that all down to regularisation?)
- Testing different model types
- Hyperparameter tuning (Manual, loops, grid search)
- Evaluating your final model **only** on the test data!
- Compare classification with a Naive baseline

If you complete everything, you might want to try the [UCI Machine learning repository](https://archive.ics.uci.edu/) to get some new datasets to practice with (for both classification and regression).

## 3.1 Exercise 1 - Breast Cancer Dataset

As the primary dataset we've been working with so far, I would like for you to take everything we've been doing above to see how good a model you can fit on the breast cancer dataset.

Guidance:
- I would like you to try fitting at least one other type of classification model other than Logistic regression.
- Try examining multiple hyperparameters.
- Try including all of the feature data in the model.
- Consider the dataset and what metric you care the most about optimising
- This is quite an 'easy' classification problem, so see how good a solution you can get with a robust 60:20:20 data splitting!
- Note: Logistic regression can still use polynomial features

The best model you get will depend on what metric you decide to maximise. I chose the recall (do you know why?) and using a 60:20:20 data split I was able to get a recall of $94\%$ on the test data. Can you beat me?

In [ ]:
##############################################################
# Your code here
##############################################################

#####################
# Load and split data
#####################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
# note here I'm selecting all of the features
columns = X_pd.columns.tolist()
X_raw = np.array(X_pd[columns])
# Note that in the dataset 0 = Malignant and 1 = Benign, but we're going to switch that around going forward
# so that the 'positive' class Malignant = 1
y = np.array(1.0-y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=1/5, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=0)


###################################
# Initialise Model and preprocessors
###################################

C_indices = np.arange(20) # Get a list of 1-20
Cs = 10.0 ** ( C_indices-10) # Define a list of Cs by taking indices from 10 to -10
# Get the recall score arrays
# Note that we could be monitoring any of the other metrics to choose our model.
# I have chosen recall as for a cancer case you really want to avoid false negatives
met_train_array = np.full([len(Cs)], np.nan)
met_valid_array = np.full([len(Cs)], np.nan)

#Initialise our scaler object - remember that for Logistic regression we should always scale features
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_valid = scaler.transform(X_valid_raw)

# Loop to test C values
for C_index, C in zip(C_indices, Cs):
  # train
  model = LogisticRegression(C=C) # fits our Logistic Regression model with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  met_train = recall_score(y_train, y_pred_train)

  # Validation
  y_pred_valid = model.predict(X_valid)
  met_valid = recall_score(y_valid, y_pred_valid)

  # Store metric values for this iteration
  met_train_array[C_index] = met_train
  met_valid_array[C_index] = met_valid
  print(f'C: {C}, Recall score: {f1_valid}.')

# Plot our graph of f1 for training and validation
plt.plot(Cs, met_train_array, label='Precision score on training dataset')
plt.plot(Cs, met_valid_array, label='Precision Score on validation dataset')
plt.xlabel(r'Regularization weights $C$')
plt.ylabel('Recall score')
plt.title(r'Regularization weights $C$ and Recall Score')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing C (highest f1)
best_c_index = np.nanargmax(met_valid_array)
best_C = Cs[best_c_index]
print(f'\nThe best C:', best_C)

##########
# Evaluate best model on the validation set
##########

model = LogisticRegression(C=best_C) # fits our Logistic Regression with a best regularisation
model.fit(X_train, y_train)
# Predict validation data
y_pred_valid = model.predict(X_valid)
# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid)))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid)))
print('F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))


In [ ]:
##############################################################
# Your code here
##############################################################

#####################
# Load and split data
#####################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
# note here I'm selecting all of the features
columns = X_pd.columns.tolist()
X_raw = np.array(X_pd[columns])
# Note that in the dataset 0 = Malignant and 1 = Benign, but we're going to switch that around going forward
# so that the 'positive' class Malignant = 1
y = np.array(1.0-y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=1/5, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=0)


###################################
# Initialise Model and preprocessors
###################################

k_indices = np.arange(10) # Get a list of 1-10
ks = [2,3,4,5,8,10,15,20,25,30] # Define a list of ks by taking indices from 10 to -10
# Get the score arrays
# Note that we could be monitoring any of the other metrics to choose our model.
# I have chosen recall as for a cancer case you really want to avoid false negatives
met_train_array = np.full([len(ks)], np.nan)
met_valid_array = np.full([len(ks)], np.nan)

#Feature Scaling
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_valid = scaler.transform(X_valid_raw)

# Loop to test C values
for k_index, k in zip(k_indices, ks):
  # train
  model = KNeighborsClassifier(n_neighbors=k) # fits our Knn with new k each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  met_train = recall_score(y_train, y_pred_train)

  # Validation
  y_pred_valid = model.predict(X_valid)
  met_valid = recall_score(y_valid, y_pred_valid)

  # Store metric values for this iteration
  met_train_array[k_index] = met_train
  met_valid_array[k_index] = met_valid
  print(f'K: {k}, Recall score: {met_valid}.')

# Plot our graph of MSE for training and validation
plt.plot(ks, met_train_array, label='Precision score on training dataset')
plt.plot(ks, met_valid_array, label='Precision Score on validation dataset')
plt.xlabel(r'Neighbours $k$')
plt.ylabel('Recall score')
plt.title(r'Neighbours $k$ and Recall Score')
plt.legend()
plt.show()

# Select our best performing k (highest f1)
best_k_index = np.nanargmax(met_valid_array)
best_k = ks[best_k_index]
print(f'\nThe best k:', best_k)


##########
# Evaluate best model on the validation set
##########

model = KNeighborsClassifier(n_neighbors=best_k)  # fits our Knn model with best k
model.fit(X_train, y_train)
# Predict validation data
y_pred_valid = model.predict(X_valid)
# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid)))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid)))
print('F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))


In [ ]:
# Looking at our results above, we can see that:
# Logistic Regression has
# Recall 0.9556
# f1 0.9053

# Knn has
# Recall 0.9333
# f1 0.9655

# (of course you can consider the other metrics as well)
# We were optimising for recall as the most important factor for cancer detection, so for this scenario I would consider the logistic regression model best.
# Also, despite what the testing says, I wouldn't really trust a Knn model using k=2
# This seems like it is still very much in danger of overfitting the validation data and being unable to generalise.

In [ ]:
# now we want to evalaute our final model!
######
# Test
######
model = LogisticRegression(C=best_C) # fits our LogisticRegression model best reg weight
model.fit(X_train, y_train)
# Preprocess test data
X_test = scaler.transform(X_test_raw)
# Predict test data
y_pred_test = model.predict(X_test)

##########
# Evaluate
##########

# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# Accuracy metrics - 1 is an perfect prediction
print('\nAccuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test)))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test)))
print('F1 Score: {:.4f}'.format(f1_score(y_test, y_test)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'\nConfusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(f'\nClassification Report:')
print(sklearn.metrics.classification_report(y_test, y_pred_test))


##3.2 Exercise 2 - Iris Dataset

Please take everything we've been doing and see how good a model you can fit on the multi-class Iris dataset.

Guidance:
- I would like you to try fitting at least one other type of classification model other than Logistic regression.
- Try examining multiple hyperparameters.
- Try including all of the data in the model.
- Note: Logistic regression can still use polynomial features

In [ ]:
##############################################################
# Your code here
##############################################################

#####################
# Load and split data
#####################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_iris(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
# note here I'm selecting all of the features
columns = X_pd.columns.tolist()
X_raw = np.array(X_pd[columns])
y = np.array(y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=1/5, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=0)


###################################
# Initialise Model and preprocessors
###################################

C_indices = np.arange(20) # Get a list of 1-20
Cs = 10.0 ** ( C_indices-10) # Define a list of Cs by taking indices from 10 to -10
# Get the recall score arrays
# Note that we cold be monitoring any of the other metrics to choose our model.
# I have chosen recall as for a cancer case you really want to avoid false negatives
met_train_array = np.full([len(Cs)], np.nan)
met_valid_array = np.full([len(Cs)], np.nan)

#Initialise our scaler object - remember that for Logistic regression we should always scale features
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_valid = scaler.transform(X_valid_raw)

# Loop to test C values
for C_index, C in zip(C_indices, Cs):
  # train
  model = LogisticRegression(C=C) # fits our Logistic Regression model with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  met_train = f1_score(y_train, y_pred_train, average='macro')

  # Validation
  y_pred_valid = model.predict(X_valid)
  met_valid = f1_score(y_valid, y_pred_valid, average='macro')

  # Store metric values for this iteration
  met_train_array[C_index] = met_train
  met_valid_array[C_index] = met_valid
  print(f'C: {C}, f1 score: {met_valid}.')

# Plot our graph of f1 for training and validation
plt.plot(Cs, met_train_array, label='f1 score on training dataset')
plt.plot(Cs, met_valid_array, label='f1 Score on validation dataset')
plt.xlabel(r'Regularization weights $C$')
plt.ylabel('f1 score')
plt.title(r'Regularization weights $C$ and f1 Score')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing C (highest f1)
best_c_index = np.nanargmax(met_valid_array)
best_C = Cs[best_c_index]
print(f'\nThe best C:', best_C)

##########
# Evaluate best model on the validation set
##########

model = LogisticRegression(C=best_C) # fits our Logistic Regression with a best regularisation
model.fit(X_train, y_train)
# Predict validation data
y_pred_valid = model.predict(X_valid)
# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid, average='macro')))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid, average='macro')))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))

In [ ]:
##############################################################
# Your code here
##############################################################
 # lets try a decision tree this time

from sklearn.tree import DecisionTreeClassifier
#####################
# Load and split data
#####################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_iris(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
# note here I'm selecting all of the features
columns = X_pd.columns.tolist()
X_raw = np.array(X_pd[columns])
# Note that in the dataset 0 = Malignant and 1 = Benign, but we're going to switch that around going forward
# so that the 'positive' class Malignant = 1
y = np.array(y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=1/5, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=0)


###################################
# Initialise Model and preprocessors
###################################

# Here I'm going to only experiment with the max tree depth, but remember there are a lot of other decision tree parameters to consider (perhaps use a grid search or some nested loops)
# I've left the search parameter as C to make my life easy, but you could update the names to be more representative
C_indices = np.arange(10) # Get a list of 1-10
Cs = [2,3,5,8,10,15,20,25,30,50] # Define a list of Cs by taking indices from 10 to -10
# Get the f1 score arrays
# Note that we cold be monitoring any of the other metrics to choose our model.
# I have chosen f1 as for this case I don't think we particularly care about minimising recall or precision specifically.
met_train_array = np.full([len(Cs)], np.nan)
met_valid_array = np.full([len(Cs)], np.nan)

#scaling not needed
# scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
X_train = X_train_raw
X_valid = X_valid_raw

# Loop to test C values
for C_index, C in zip(C_indices, Cs):
  # train
  model = DecisionTreeClassifier(max_depth = C) # fits our Decision tree model with a different max depth
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  met_train = f1_score(y_train, y_pred_train, average='macro')

  # Validation
  y_pred_valid = model.predict(X_valid)
  met_valid = f1_score(y_valid, y_pred_valid, average='macro')

  # Store metric values for this iteration
  met_train_array[C_index] = met_train
  met_valid_array[C_index] = met_valid
  print(f'C: {C}, f1 score: {f1_valid}.')

# Plot our graph of f1 for training and validation
plt.plot(Cs, met_train_array, label='f1 score on training dataset')
plt.plot(Cs, met_valid_array, label='f1 Score on validation dataset')
plt.xlabel(r'Max Tree Depth $C$')
plt.ylabel('f1 score')
plt.title(r'Max Tree Depth $C$ and f1 Score')
plt.legend()
plt.show()

# Select our best performing C (highest f1)
best_c_index = np.nanargmax(met_valid_array)
best_C = Cs[best_c_index]
print(f'\nThe best C:', best_C)

##########
# Evaluate best model on the validation set
##########

model = DecisionTreeClassifier(max_depth = best_C) # fits our Logistic Regression with a best regularisation
model.fit(X_train, y_train)
# Predict validation data
y_pred_valid = model.predict(X_valid)
# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid, average='macro')))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid, average='macro')))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))

In [ ]:
# Possibly we can get a better result with the decision tree with greater tuning of the parameters.
# But this is a good case to show how seriously decision trees often overfit the training data.

#feel free to explore the decision tree more, but based on these results we can clearly see we're better off with the logistic regression model
#where C=1

In [ ]:
# now we want to evaluate our final model!
######
# Test
######
model = LogisticRegression(C=1) # fits our ridge model with a new regularisation weight each iteration
model.fit(X_train, y_train)
# Preprocess test data
X_test = scaler.transform(X_test_raw)
# Predict test data
y_pred_test = model.predict(X_test)

##########
# Evaluate
##########

# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# Accuracy metrics - 1 is an perfect prediction
print('\nAccuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test, average='macro')))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test, average='macro')))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'\nConfusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(f'\nClassification Report:')
print(sklearn.metrics.classification_report(y_test, y_pred_test))


##3.3 Exercise 3 - Nonlinear Data

Here I have used the `make_moons` command to create a dataset that has a very nonlinear relationship between the two features (but has clear groups visually).

Can you fit a model that classifies this nonlinear data well?

Extra: Can you find a way to make Logistic regression classify this dataset?

In [ ]:
from sklearn.datasets import make_moons

# Create a synthetic dataset with two interleaving half circles
X, y = make_moons(n_samples=300, noise=0.15, random_state=42)

# Plot the decision boundary
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title('Synthetic Nonlinear Data')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

In [ ]:
##############################################################
# Your code here
##############################################################
#####################
# Load and split data
#####################

# Lets start by just using a normal logistic regression
# We'll see that it doesn't behave very well...

# Create a synthetic dataset with two interleaving half circles
X, y = make_moons(n_samples=300, noise=0.15, random_state=42)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X, y, test_size=1/5, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=0)


###################################
# Initialise Model and preprocessors
###################################

C_indices = np.arange(20) # Get a list of 1-20
Cs = 10.0 ** ( C_indices-10) # Define a list of Cs by taking indices from 10 to -10
# Get the F1 score arrays
# Note that we could be monitoring any of the other metrics to choose our model.
#I have chosen f1 here as it give a balanced penalisation of accuracy and recall, but for your case you may care about maximising another metric
f1_train_array = np.full([len(Cs)], np.nan)
f1_valid_array = np.full([len(Cs)], np.nan)

#Initialise our scaler object - remember that for Logistic regression we should always scale features
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_valid = scaler.transform(X_valid_raw)

# Loop to test C values
for C_index, C in zip(C_indices, Cs):
  # train
  model = LogisticRegression(C=C) # fits our Logistic Regression model with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  f1_train = f1_score(y_train, y_pred_train)

  # Validation
  y_pred_valid = model.predict(X_valid)
  f1_valid = f1_score(y_valid, y_pred_valid)

  # Store f1s for this iteration
  f1_train_array[C_index] = f1_train
  f1_valid_array[C_index] = f1_valid
  print(f'C: {C}, f1 score: {f1_valid}.')

# Plot our graph of F1 for training and validation
plt.plot(Cs, f1_train_array, label='F1 score on training dataset')
plt.plot(Cs, f1_valid_array, label='F1 Score on validation dataset')
plt.xlabel(r'Regularization weights $C$')
plt.ylabel('F1 Score')
plt.title(r'Regularization weights $C$ and F1 Score')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing C (highest f1)
best_c_index = np.nanargmax(f1_valid_array)
best_C = Cs[best_c_index]
print(f'\nThe best C:', best_C)

##########
# Evaluate best model on the validation set
##########

model = LogisticRegression(C=best_C) # fits our Logistic Regression with a best regularisation
model.fit(X_train, y_train)
# Predict validation data
y_pred_valid = model.predict(X_valid)

# Plot the decision boundary

h = .02  # step size in the mesh
x_min, x_max = X_valid[:, 0].min() - 1, X_valid[:, 0].max() + 1
y_min, y_max = X_valid[:, 1].min() - 1, X_valid[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
# Plot the points
plt.scatter(X_valid[:, 0], X_valid[:, 1], c=y_valid, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title('Synthetic Classification Problem using SVM')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid)))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid)))
print('F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))



In [ ]:
#####################
# Load and split data
################
# One way around this is to bring back our polynomial features!
from sklearn.preprocessing import PolynomialFeatures

# Create a synthetic dataset with two interleaving half circles
X, y = make_moons(n_samples=300, noise=0.15, random_state=42)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X, y, test_size=1/5, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=0)


###################################
# Initialise Model and preprocessors
###################################
degree = 3 # select some polynomial degree
C_indices = np.arange(20) # Get a list of 1-20
Cs = 10.0 ** ( C_indices-10) # Define a list of Cs by taking indices from 10 to -10
# Get the F1 score arrays
# Note that we could be monitoring any of the other metrics to choose our model.
#I have chosen f1 here as it give a balanced penalisation of accuracy and recall, but for your case you may care about maximising another metric
f1_train_array = np.full([len(Cs)], np.nan)
f1_valid_array = np.full([len(Cs)], np.nan)

#Initialise our scaler object - remember that for Logistic regression we should always scale features
##### Add Poly object############
poly = PolynomialFeatures(degree=degree)
################################
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
poly.fit(X_train_raw)
X_train_poly = poly.transform(X_train_raw)
scaler.fit(X_train_poly)
X_train = scaler.transform(X_train_poly)
X_valid_poly = poly.transform(X_valid_raw)
X_valid = scaler.transform(X_valid_poly)

# Loop to test C values
for C_index, C in zip(C_indices, Cs):
  # train
  model = LogisticRegression(C=C) # fits our Logistic Regression model with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  f1_train = f1_score(y_train, y_pred_train)

  # Validation
  y_pred_valid = model.predict(X_valid)
  f1_valid = f1_score(y_valid, y_pred_valid)

  # Store f1s for this iteration
  f1_train_array[C_index] = f1_train
  f1_valid_array[C_index] = f1_valid
  print(f'C: {C}, f1 score: {f1_valid}.')

# Plot our graph of F1 for training and validation
plt.plot(Cs, f1_train_array, label='F1 score on training dataset')
plt.plot(Cs, f1_valid_array, label='F1 Score on validation dataset')
plt.xlabel(r'Regularization weights $C$')
plt.ylabel('F1 Score')
plt.title(r'Regularization weights $C$ and F1 Score')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing C (highest f1)
best_c_index = np.nanargmax(f1_valid_array)
best_C = Cs[best_c_index]
print(f'\nThe best C:', best_C)

##########
# Evaluate best model on the validation set
##########



model = LogisticRegression(C=best_C) # fits our Logistic Regression with a best regularisation
model.fit(X_train, y_train)
# Predict validation data
y_pred_valid = model.predict(X_valid)

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid)))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid)))
print('F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))




In [ ]:
#####################
# Load and split data
#####################

# Alternate solution - a Support Vector Machine!
from sklearn.svm import SVC

# Create a synthetic dataset with two interleaving half circles
X, y = make_moons(n_samples=300, noise=0.15, random_state=42)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X, y, test_size=1/5, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=0)


###################################
# Initialise Model and preprocessors
###################################

C_indices = np.arange(20) # Get a list of 1-20
Cs = 10.0 ** ( C_indices-10) # Define a list of Cs by taking indices from 10 to -10
# Get the F1 score arrays
# Note that we could be monitoring any of the other metrics to choose our model.
#I have chosen f1 here as it give a balanced penalisation of accuracy and recall, but for your case you may care about maximising another metric
f1_train_array = np.full([len(Cs)], np.nan)
f1_valid_array = np.full([len(Cs)], np.nan)

#Initialise our scaler object - remember that for SVM we should always scale features
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_valid = scaler.transform(X_valid_raw)

# Loop to test C values
for C_index, C in zip(C_indices, Cs):
  # train
  model = SVC(C=C) # fits our SVM with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  f1_train = f1_score(y_train, y_pred_train)

  # Validation
  y_pred_valid = model.predict(X_valid)
  f1_valid = f1_score(y_valid, y_pred_valid)

  # Store f1s for this iteration
  f1_train_array[C_index] = f1_train
  f1_valid_array[C_index] = f1_valid
  print(f'C: {C}, f1 score: {f1_valid}.')

# Plot our graph of F1 for training and validation
plt.plot(Cs, f1_train_array, label='F1 score on training dataset')
plt.plot(Cs, f1_valid_array, label='F1 Score on validation dataset')
plt.xlabel(r'Regularization weights $C$')
plt.ylabel('F1 Score')
plt.title(r'Regularization weights $C$ and F1 Score')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing C (highest f1)
best_c_index = np.nanargmax(f1_valid_array)
best_C = Cs[best_c_index]
print(f'\nThe best C:', best_C)

##########
# Evaluate best model on the validation set
##########

model = SVC(C=best_C) # fits our SVM with a best regularisation
model.fit(X_train, y_train)
# Predict validation data
y_pred_valid = model.predict(X_valid)

# Plot the decision boundary

h = .02  # step size in the mesh
x_min, x_max = X_valid[:, 0].min() - 1, X_valid[:, 0].max() + 1
y_min, y_max = X_valid[:, 1].min() - 1, X_valid[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)
# Plot the points
plt.scatter(X_valid[:, 0], X_valid[:, 1], c=y_valid, cmap='viridis', edgecolors='k', marker='o', s=50, linewidth=1.5)
plt.title('Synthetic Classification Problem using SVM')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid)))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid)))
print('F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))



In [ ]:
# This is a case where both methods have given us identical answers! A SVM with an rbf kernel like this could be equivalent to logistic regression in some cases
# as the rbf essentially fits infiite polynomial features to find the best comination. Given that this is quite simple data it could be that degree 3 is the optimal degree anyway.

# For this case I'm going to go with our logistic regression model as we have a clearere idea of what is happening with the features, but picking the SVM is entirely justifiable.

In [ ]:
# now we want to evaluate our final model!
######
# Test
######
model = LogisticRegression(C=10000000.0) # fits our ridge model with a new regularisation weight each iteration
model.fit(X_train, y_train)
# Preprocess test data - remember we need poly features for this case.
poly.fit(X_train_raw)
X_train_poly = poly.transform(X_train_raw)
scaler.fit(X_train_poly)
X_train = scaler.transform(X_train_poly)
X_test_poly = poly.transform(X_test_raw)
X_test = scaler.transform(X_test_poly)
# Predict test data
y_pred_test = model.predict(X_test)

##########
# Evaluate
##########

# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# Accuracy metrics - 1 is an perfect prediction
print('\nAccuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test)))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test)))
print('F1 Score: {:.4f}'.format(f1_score(y_test, y_test)))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'\nConfusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat, display_labels=['benign', 'malignant']).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(f'\nClassification Report:')
print(sklearn.metrics.classification_report(y_test, y_pred_test))


##3.4 Exercise 4 - Wine Dataset

This is another multi class classification dataset. See how accurate a solution you can get to this previously unseen dataset.

Guidance:
- I would like you to try fitting at least one other type of classification model other than Logistic regression.
- Try examining multiple hyperparameters.
- Try including all of the data in the model.

In [ ]:
##############################################################
# Your code here
##############################################################

#####################
# Load and split data
#####################
# Load the wine dataset
X_pd, y_pd = sklearn.datasets.load_wine(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
# note here I'm selecting all of the features
columns = X_pd.columns.tolist()
X_raw = np.array(X_pd[columns])

y = np.array(y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=1/5, shuffle=True, random_state=1337)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=1337)


###################################
# Initialise Model and preprocessors
###################################

C_indices = np.arange(20) # Get a list of 1-20
Cs = 10.0 ** ( C_indices-10) # Define a list of Cs by taking indices from 10 to -10
# Get the recall score arrays
# Note that we cold be monitoring any of the other metrics to choose our model.
# I have chosen recall as for a cancer case you really want to avoid false negatives
met_train_array = np.full([len(Cs)], np.nan)
met_valid_array = np.full([len(Cs)], np.nan)

#Initialise our scaler object - remember that for Logistic regression we should always scale features
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_valid = scaler.transform(X_valid_raw)

# Loop to test C values
for C_index, C in zip(C_indices, Cs):
  # train
  model = LogisticRegression(C=C) # fits our Logistic Regression model with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  met_train = f1_score(y_train, y_pred_train, average='macro')

  # Validation
  y_pred_valid = model.predict(X_valid)
  met_valid = f1_score(y_valid, y_pred_valid, average='macro')

  # Store metric values for this iteration
  met_train_array[C_index] = met_train
  met_valid_array[C_index] = met_valid
  print(f'C: {C}, f1 score: {met_valid}.')

# Plot our graph of f1 for training and validation
plt.plot(Cs, met_train_array, label='f1 score on training dataset')
plt.plot(Cs, met_valid_array, label='f1 Score on validation dataset')
plt.xlabel(r'Regularization weights $C$')
plt.ylabel('f1 score')
plt.title(r'Regularization weights $C$ and f1 Score')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing C (highest f1)
best_c_index = np.nanargmax(met_valid_array)
best_C = Cs[best_c_index]
print(f'\nThe best C:', best_C)

##########
# Evaluate best model on the validation set
##########

model = LogisticRegression(C=best_C) # fits our Logistic Regression with a best regularisation
model.fit(X_train, y_train)
# Predict validation data
y_pred_valid = model.predict(X_valid)
# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# The accuracy score: 1 for perfect prediction
print('Accuracy: {:.4f}'.format(accuracy_score(y_valid, y_pred_valid)))
print('Precision: {:.4f}'.format(precision_score(y_valid, y_pred_valid, average='macro')))
print('Recall: {:.4f}'.format(recall_score(y_valid, y_pred_valid, average='macro')))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_valid, y_pred_valid)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_valid, y_pred_valid, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_valid, y_pred_valid, normalize='all')
print(f'Confusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
# Note, the Precision/Recall/F1 in the report match the positive class (1.0) in the report
print(sklearn.metrics.classification_report(y_valid, y_pred_valid))

In [ ]:
# Frankly this is looking pretty good already, but let's try another model

##############################################################
# Your code here
##############################################################
 # lets try a Radom Forest this time
 # Let's also try optimising with Grid Search instead!

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
#####################
# Load and split data
#####################
# Load the breast cancer dataset
X_pd, y_pd = sklearn.datasets.load_wine(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
# note here I'm selecting all of the features
columns = X_pd.columns.tolist()
X_raw = np.array(X_pd[columns])
# Note that in the dataset 0 = Malignant and 1 = Benign, but we're going to switch that around going forward
# so that the 'positive' class Malignant = 1
y = np.array(y_pd)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=1/5, shuffle=True, random_state=1337)


###################################
# Initialise Model and preprocessors
###################################

# Hyperparameter grid for GridSearchCV
param_grid = {
    'forest__n_estimators': [10, 30, 100],  # Values for n_estimators
    'forest__max_depth': [5, 10, 30],  # Values for DecisionTreeRegressor max depth
    'forest__min_samples_split': [2, 10],  # Values for DecisionTreeRegressor min_samples_split
    'forest__min_samples_leaf': [2, 10],  # Values for DecisionTreeRegressor min_samples_split
}

# Initialize Model (no preprocessing needed)
forest = RandomForestClassifier(random_state=59)

# Create a pipeline with preprocessing and Decision Tree regression
pipeline = Pipeline([
    ('forest', forest)
])

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model with GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_

# Print the best model
print("Best Model:", best_model)


In [ ]:
# Get the results as a DataFrame
cv_results = pd.DataFrame(grid_search.cv_results_)

# Display the relevant columns
relevant_columns = ['params', 'mean_test_score', 'std_test_score']
cv_results[relevant_columns].sort_values(by='mean_test_score', ascending=False)


In [ ]:
# now we want to evalaute our final model!
######
# Test
######
best_model = grid_search.best_estimator_
#best_model = RandomForestClassifier(n_estimators=100, max_depth = 5, min_samples_split = 10, min_samples_leaf = 10, random_state=59)
# Train the best model on the full training set
best_model.fit(X_train, y_train)
# Predict test data
y_pred_test = best_model.predict(X_test)

##########
# Evaluate
##########

# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# Accuracy metrics - 1 is an perfect prediction
print('\nAccuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test, average='macro')))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test, average='macro')))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'\nConfusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(f'\nClassification Report:')
print(sklearn.metrics.classification_report(y_test, y_pred_test))


In [ ]:
# While the Random Forest does quite well, the logistic regression is clearly winning.
# This could be an issue of Decision Trees (and hence random forests) not extrapolating well

# Essentially, we're using quite a quite large testing set, and our model looks like it may be overfitting our training data a bit.
# rerunning the grid search with parameters set in ranges that will make it harder to overfit (shallower models, more datapoints to split, etc) may help
# Also may be worth increasing the amount of training data used.

# Regardless, logistic regression is going a good job with how things are set up now, so let's evaluate it!

In [ ]:
# now we want to evaluate our final model!
######
# Test
######

# Make sure our data is setup and processed properly, as the Random Forest did things a bit differently
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=1/5, shuffle=True, random_state=1337)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=1/4, shuffle=True, random_state=1337)

scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)

model = LogisticRegression(C=best_C) # fits our ridge model with a new regularisation weight each iteration
model.fit(X_train, y_train)
# Preprocess test data
X_test = scaler.transform(X_test_raw)
# Predict test data
y_pred_test = model.predict(X_test)

##########
# Evaluate
##########

# Note, because this is multivariable classification I've not included a scatter plot of the decision surface.

# Accuracy metrics - 1 is an perfect prediction
print('\nAccuracy: {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
print('Precision: {:.4f}'.format(precision_score(y_test, y_pred_test, average='macro')))
print('Recall: {:.4f}'.format(recall_score(y_test, y_pred_test, average='macro')))
print('Balanced Accuracy: {:.4f}'.format(balanced_accuracy_score(y_test, y_pred_test)))
print('Macro averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='macro')))
print('Weighted averaged F1 Score: {:.4f}'.format(f1_score(y_test, y_pred_test, average='weighted')))
# Confusion matrix
confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred_test, normalize='all')
print(f'\nConfusion matrix: \n', confusion_mat)
# Visualize the confusion matrix
sklearn.metrics.ConfusionMatrixDisplay(confusion_mat).plot(cmap=plt.cm.Blues)
plt.grid(False)
# The classification report, which contains accuracy, precision, recall, F1 score
print(f'\nClassification Report:')
print(sklearn.metrics.classification_report(y_test, y_pred_test))
